Lo scopo è una più completa associazione tra i CIG e i CUP

Nel notebook sono svolte le seguenti op:
1) Fare merge tra "P03" (misure premiali) e "P01" (associazione CIG-CUP), ne risulta dataset "anac_13".
2) Prendere un subset di "anac_13" con le righe in cui i CIG sono privi di CUP, ne risulta dataset "anac_13_unmatched".
3) Fare merge tra P06 (Gare ReGis) con "anac_13_unmatched" per vedere se alcuni CIG hanno un corrispondente CUP in p06, ne risulta che un sottoinsieme di CIG orfani ha CUP in P06.
4) Fare merge tra P03 con P06 (Gare ReGis)

In [1]:
# dependencies
import pandas as pd
import numpy as np

In [2]:
# datasets
p01 = pd.read_csv("./datasets/cup_csv.csv",delimiter=';')
p03 = pd.read_excel("./datasets/p06-p03-p04_formattati.xlsx", sheet_name="misurepremiali-pnrrpnc_csv")
p04 = pd.read_excel("./datasets/p06-p03-p04_formattati.xlsx", sheet_name="indicatori-pnrrpnc_csv")
p06 = pd.read_excel("./datasets/p06-p03-p04_formattati.xlsx", sheet_name="PNRR_Gare-Dati_Validati")

In [3]:
# preprocessing
p01 = p01[["CIG", "CUP"]]
p03 = p03.rename(columns={'cig':'CIG'})
p04 = p04.rename(columns={'cig':'CIG'})

In [4]:
# merging anac dataset p01 (associazione cig-cup) e p03 (premialità)
anac_13 = p03.merge(p01, on='CIG',how='left')
print('CIG privi di CUP:',anac_13[anac_13['CUP'].isna()]['CIG'].nunique())
print('Esempi di CIG privi di CUP:',anac_13[anac_13['CUP'].isna()]['CIG'].unique())

CIG privi di CUP: 8453
Esempi di CIG privi di CUP: ['9976140B27' '996699614C' '99818578F9' ... 'A00A492F95' 'A01CB2FEF6'
 'A0149545FB']


In [140]:
# esempio di CIG privo di CUP
anac_13[anac_13['CIG']=='9234507C3D']

,CIG,cod_mis_premiale,misura_premiale,CUP
7382,9234507C3D,1,NEI TRE ANNI ANTECEDENTI LA DATA DI SCADENZA D...,NaN
7383,9234507C3D,2,UTILIZZI O SI IMPEGNI A UTILIZZARE SPECIFICI S...,NaN


In [64]:
# crea dataset "anac_13_unmatched" con CIG privi di CUP, per mergiarlo con p06
anac_13_unmatched = anac_13[anac_13['CUP'].isna()]
m = anac_13_unmatched.merge(p06,on='CIG',how='left')

In [136]:
# CIG che in p06 hanno un CUP corrispondente
matching = m[m['CUP_y'].notna()]
# cig senza cup
non_matching = m[m['CUP_y'].isna()]
print(matching['CIG'].nunique(), '(CIG privi di CUP che hanno "acquisito" un CUP dopo il merge)')

1552 (CIG privi di CUP che hanno "acquisito" un CUP dopo il merge)


In [137]:
# esempi di cig senza cup che hanno cup in p06
matching['CIG']

12       9735915B36
13       9735915B36
31       9465732926
34       9735708067
35       9735708067
            ...    
18706    9757982D79
18707    9757982D79
18708    9757982D79
18844    99276392DF
18845    99276392DF
Name: CIG, Length: 3731, dtype: object

In [147]:
# esempi di cig ancora senza cup
non_matching['CIG']

0        9976140B27
1        996699614C
2        996699614C
3        996699614C
4        996699614C
            ...    
18918    A00A492F95
18919    A01CB2FEF6
18920    A0149545FB
18921    A0149545FB
18922    A0149545FB
Name: CIG, Length: 15192, dtype: object

In [142]:
# il cig di esempio risulta tra i matching now?
matching.columns[matching.isin(['9234507C3D']).any()]

Index(['CIG Accordo Quadro'], dtype='object')

In [146]:
non_matching.columns[non_matching.isin(['9234507C3D']).any()]

Index(['CIG'], dtype='object')

In [120]:
matching[matching['CIG Accordo Quadro']=='9234507C3D']['CIG'].unique()

array(['9621836E3D', '962541146F', '9620534BCC', '962102944B',
       '96039126E5', '9625829D5E', '962458177E', '9622573E6E',
       '96253924C1', '962587100B', '96259132B3', '9623336417',
       '9625801645', '9624175873', '9622505653'], dtype=object)

In [14]:
# fare merge tra p03 (premialità) e p06
merge_36 = p06.merge(p03,on='CIG',how='inner')

In [5]:
merge_36 = p03.merge(p06,on='CIG',how='inner')

In [7]:
p03['CIG'].nunique()

11402

In [8]:
merge_36['CIG'].nunique()

2215

In [9]:
merge_36.columns

Index(['CIG', 'cod_mis_premiale', 'misura_premiale',
       'Codice Univoco Submisura', 'Descrizione Submisura',
       'Descrizione Procedura di Aggiudicazione', 'CUP',
       'Codice Locale Progetto', 'CIG Accordo Quadro',
       'Codice Procedura Utente', 'Modalit� di Realizzazione',
       'Codice Interno PDA', 'Oggetto Principale del Contratto',
       'Oggetto Gara', 'Data Pubblicazione del CIG',
       'Codice Motivo Assenza CIG', 'Descrizione Motivo Assenza CIG',
       'Importo Complessivo Gara', 'Importo Aggiudicazione',
       'Data Aggiudicazione Definitiva', 'Data di Aggiornamento'],
      dtype='object')

In [10]:
merge_37 = p03.merge(p06,left_on='CIG', right_on='CIG Accordo Quadro', how='inner')

In [14]:
p06['CIG'].nunique()

56984

In [13]:
merge_37['CIG_x'].nunique()

121

In [79]:
merge_36['CIG'].nunique()

2215

In [32]:
p03.head()

,CIG,cod_mis_premiale,misura_premiale
0,9976140B27,1,NEI TRE ANNI ANTECEDENTI LA DATA DI SCADENZA D...
1,996699614C,1,NEI TRE ANNI ANTECEDENTI LA DATA DI SCADENZA D...
2,996699614C,3,"SI IMPEGNI AD ASSUMERE, OLTRE ALLA SOGLIA MINI..."
3,996699614C,4,"SI IMPEGNI AD ASSUMERE, OLTRE ALLA SOGLIA MINI..."
4,996699614C,5,"SI IMPEGNI AD ASSUMERE, OLTRE ALLA SOGLIA MINI..."


In [33]:
temp = p03.merge(p01, on='CIG',how='left')

In [36]:
temp['CUP'].notna().sum()

6607

In [25]:
temp1 = temp.merge(p06,on='CUP',how='left')

In [26]:
temp1.columns

Index(['CIG', 'cod_mis_premiale', 'misura_premiale', 'CUP_x',
       'Codice Univoco Submisura', 'Descrizione Submisura',
       'Descrizione Procedura di Aggiudicazione', 'CUP_y',
       'Codice Locale Progetto', 'CIG Accordo Quadro',
       'Codice Procedura Utente', 'Modalit� di Realizzazione',
       'Codice Interno PDA', 'Oggetto Principale del Contratto',
       'Oggetto Gara', 'Data Pubblicazione del CIG',
       'Codice Motivo Assenza CIG', 'Descrizione Motivo Assenza CIG',
       'Importo Complessivo Gara', 'Importo Aggiudicazione',
       'Data Aggiudicazione Definitiva', 'Data di Aggiornamento'],
      dtype='object')

In [38]:
print(temp1['CUP_x'].notna().sum())
temp1['CUP_y'].notna().sum()

7400


6088

In [ ]:
temp.mer